In [ ]:
import sys
sys.path.append('..')
import numpy as np
import math
import matplotlib.pylab as plt
import rti_msr_utils as rti
import heapq


In [ ]:
num_CIR_samples = 50  # number of logged CIR samples
CIR_samp_period = 1/(2*(4*124.8e6))  # (s) sampling period for CIR measurement
anchor_UWB_Ids = [0, 1, 2, 3, 4]  # device Ids
SpeedofLight = 299792458  # (m/s) speed of light

In [ ]:
anchor_pair_list = []
for i in range(0,5):
    for j in range(i+1,5):
        anchor_pair_list.append((i,j))

![image](..\pic\room1.png)

In [ ]:
# room 1
pos = [(2.28,0.9),(0,1.55),(5.09,6.52),(5.09,2.2),(0.2,5.69)]
room_length = 5.09
room_width = 10.02
rel_x = 1.25
rel_y = 1.98
# x_monitored = 2.45
# y_monitored = 4.40
x_monitored = room_length
y_monitored = room_width

In [ ]:
file_path = "../../data/uwb2/uwb2_exp001.csv"
uwb2_exp001, cir, tx_id_uwb, rx_id_uwb, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y = rti.load_csv_data(file_path, num_CIR_samples)

cir_up = rti.upsample_and_align_cir(cir, fp, freq_s_ratio=64)
grouped_data = rti.group_data_by_tx_rx(tx_id_uwb, rx_id_uwb, cir, cir_up, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y)


In [ ]:
# Constants
ALPHA_H = 0.05
BETA_B = 0.001
BETA_C = 0.1
MAX_COUNT = 1000
AMPLIFY = 1.3
WINDOW_SIZE = 8
SEGMENT_THRESHOLD = 5

In [ ]:
min_heap = []
for anchor_pair, data in grouped_data.items():
    data['h'] = None
    data['var_b'] = None
    data['var_c'] = None
    for i, time_stamp in enumerate(data['time_stamp']):
        heapq.heappush(min_heap, (time_stamp, anchor_pair, i))


In [ ]:

# Process the heap
count = 0
while min_heap:
    # Exit if reached MAX_COUNT
    if count == MAX_COUNT:
        break

    # Pop the smallest time_stamp item
    smallest_time, anchor_pair, index = heapq.heappop(min_heap)

    # Print current state (for debugging)
    print(f"Processing anchor pair {anchor_pair} with time stamp {smallest_time} at index {index}")
    print(f"Item data: {grouped_data[anchor_pair]}")

    # Update count
    count += 1

    # Extract and update variables
    z = grouped_data[anchor_pair]['cir_up'][index]
    if grouped_data[anchor_pair]['h'] is None:
        grouped_data[anchor_pair]['h'] = z
        continue

    h = grouped_data[anchor_pair]['h'] = grouped_data[anchor_pair]['h'] * (1 - ALPHA_H) + z * ALPHA_H
    if grouped_data[anchor_pair]['var_b'] is None:
        delta = np.abs(z - h)
        grouped_data[anchor_pair]['var_b'] = delta
        grouped_data[anchor_pair]['var_c'] = delta
    else:
        grouped_data[anchor_pair]['var_b'] = grouped_data[anchor_pair]['var_b'] * (1 - BETA_B) + np.abs(z - h) * BETA_B
        grouped_data[anchor_pair]['var_c'] = grouped_data[anchor_pair]['var_c'] * (1 - BETA_C) + np.abs(z - h) * BETA_C    

    # Extract and preprocess data
    variance = grouped_data[anchor_pair]['var_c'] - AMPLIFY * grouped_data[anchor_pair]['var_b']
    data_length = len(variance)

    # Initialize position marker
    position_marker = -1  # Position where the condition is met

    # Define loop limit to avoid redundant calculations
    loop_limit = data_length - WINDOW_SIZE + 1

    # Search for the position where the condition is met
    for j in range(data_length):
        # Skip negative variance values
        if variance[j] < 0:
            continue

        # Avoid going out of bounds
        if j >= loop_limit:
            break

        # Calculate sum of the window and check against threshold
        window_sum = sum(variance[j:j + WINDOW_SIZE])
        if window_sum >= SEGMENT_THRESHOLD:
            position_marker = j
            break

    # Position marker now holds the first position where the condition is met or -1 if it's not met
print("Done!")

In [ ]:
grouped_data[anchor_pair]['var_b']

In [ ]:
plt.plot(grouped_data[anchor_pair]['var_b'])
plt.plot(grouped_data[anchor_pair]['var_c'])

variance = grouped_data[anchor_pair]['var_c'] - 1.3*grouped_data[anchor_pair]['var_b']
plt.plot(variance)